In [ ]:
!pip install pytorch-msssim

In [ ]:
import torch
from torch import nn, tensor
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision import transforms
import math
import matplotlib.pyplot as plt
import cv2
from skimage.measure import compare_ssim
import numpy as np
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
drive_path = "/content/drive/MyDrive"
train_dataset = TensorDataset(
    torch.load(f"{drive_path}/imc-prediction/tensors/inputs.pt"),
    torch.load(f"{drive_path}/imc-prediction/tensors/collagen_targets.pt"))

In [ ]:
img1, img2 = next(iter(train_dataset))

Test SSMI loss

In [ ]:
from torch.autograd import Variable
img1 = Variable( img1,  requires_grad=False)
img2 = Variable( img2,  requires_grad=True)
ssim_value = ssim(img1.unsqueeze(0), img2.unsqueeze(0)).item()
print("Initial ssim:", ssim_value)

ssim_loss = SSIM(win_size=11, win_sigma=1.5, data_range=1, size_average=True, channel=1)

optimizer = torch.optim.Adam([img2], lr=0.01)
i = 0
while i < 10:
    optimizer.zero_grad()
    _ssim_loss = 1-ssim_loss(img1.unsqueeze(0), img2.unsqueeze(0))
    _ssim_loss.backward()
    optimizer.step()

    ssim_value = ssim(img1.unsqueeze(0), img2.unsqueeze(0)).item()
    print(ssim_value)
    i += 1

Initial ssim: 0.7615805864334106
0.7662784457206726
0.7903226613998413
0.8022977709770203
0.8115919232368469
0.8250197768211365
0.8385151028633118
0.8488903045654297
0.8571313619613647
0.8658400177955627
0.8753970861434937


Peak signal-to-noise ratio

In [ ]:
def compute_psnr(img1, img2):
  mse = torch.mean(torch.square(img1 - img2))
  return 10 * math.log10(1. / mse)

In [ ]:
compute_psnr(data, label)

12.906139854625646

Frechet Inception distance

In [ ]:
import scipy
def frechet(img1, img2):
  img1 = np.array(img1.squeeze())
  img2 = np.array(img2.squeeze())
  mu1, sigma1 = img1.mean(axis=0), np.cov(img1, rowvar=False)
  mu2, sigma2 = img2.mean(axis=0), np.cov(img2, rowvar=False)
  ssdiff = np.sum((mu1 - mu2)**2.0)
  covmean = scipy.linalg.sqrtm(sigma1.dot(sigma2))
  if np.iscomplexobj(covmean):
    covmean = covmean.real
  fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

In [ ]:
frechet(data, label)

9.896095821643893